# Overview of **SlideRule** Functionality

[SlideRule](https://slideruleearth.io/web/) is a web service that provides on-demand customized data products.  The primary way of accessing SlideRule is through its Python client.

This notebook gives an overview of the different functionality SlideRule provides. 

```{admonition} Learning Objectives
- How to import and configure the SlideRule Python client
- Survey SlideRule's core and advanced functionality
- Where to find documentation on SlideRule's APIs
```

## Import and Configure SlideRule

The SlideRule Python client currently consists of seven primary modules.

* __sliderule__ - the core module
* __earthdata__ - functions that access CMR (NASA's Common Metadata Repository), CMR-STAC, and TNM (The National Map, for the 3DEP data hosted by USGS)
* __h5__ - APIs for directly reading HDF5 and NetCD4 data
* __raster__ - APIs for sampling supported raster datasets
* __icesat2__ - APIs for processing ICESat-2 data
* __gedi__ - APIs for processing GEDI data
* __io__ - functions for reading and writing local files with SlideRule results
* __ipysliderule__ - functions for building interactive Jupyter notebooks that interface to SlideRule

These modules can be imported into your environment like so:

In [1]:
from sliderule import sliderule, earthdata, h5, raster, icesat2, gedi

/srv/conda/envs/notebook/lib/python3.10/site-packages/sliderule/sliderule.py:41: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas


Once those modules are imported, the next thing you'll likely want to do is configure the client with the settings you want to use.  That is done with a call to `sliderule.init()`.  For detailed documentation on what arguments are supported by the initialization function, check out the [api reference page](https://slideruleearth.io/web/rtd/api_reference/sliderule.html#init).

It is not necessary to call `sliderule.init()` in order to start using the client, since the default settings provide a working system.  Nevertheless, it is a good practice to include a call to this function early in your notebook as a placeholder for when different settings are desired.  For instance, if you want to change the verbosity of the client and enable logging to the console, you can do so as shown below.

In [2]:
sliderule.init(verbose=True)

## Core Functionality
- [Directly read HDF5 and NetCDF4 files](#h5p)
- [Subset ATL03 photon cloud data](#atl03p)
- Generate customized ATL06 elevations
- Generate customized ATL06 elevations using ATL08 classifications
- Generate customized ATL08 vegetation metrics
- Sample rasters (HLS, ArcticDEM, REMA, 3DEP, GEDI L3/L4A) at points of interest
- Subset GEDI L1B, L2A, L4A

<a id='h5p'></a>
### Directly read HDF5 and NetCDF files

The **h5** module provides APIs for directly reading HDF5 and NetCDF4 files hosted by NASA in the cloud. [`h5.h5p`](https://slideruleearth.io/web/rtd/api_reference/h5.html#h5p) is the primary method used to directly read data in the cloud.  The [reference page](https://slideruleearth.io/web/rtd/api_reference/h5.html#h5p) for `h5.h5p` provides a description of each of the arguments needed to make the call and the different options available.  


```{tip}
Under-the-hood, the functions in the **h5** module make HTTP requests to SlideRule servers running in us-west-2, and those servers read the requested data from S3 and return the results in an HTTP response back to the client.
```

In the example below, the first 100 latitudes and longitudes are read from an ATL06 granule.  The results are returned in a dictionary of numpy arrays, where each key is the name of the dataset.  If instead of reading just the first 100 values, all the values need to be read, then "numrows" can be set to `h5.ALL_ROWS`. 

In [ ]:
asset = "icesat2"
resource="ATL06_20181017222812_02950102_006_02.h5"
datasets = [
    # latitudes
    {"dataset": "/gt1l/land_ice_segments/latitude", "startrow": 0, "numrows": 100},
    {"dataset": "/gt1r/land_ice_segments/latitude", "startrow": 0, "numrows": 100},
    {"dataset": "/gt2l/land_ice_segments/latitude", "startrow": 0, "numrows": 100},
    {"dataset": "/gt2r/land_ice_segments/latitude", "startrow": 0, "numrows": 100},
    {"dataset": "/gt3l/land_ice_segments/latitude", "startrow": 0, "numrows": 100},
    {"dataset": "/gt3r/land_ice_segments/latitude", "startrow": 0, "numrows": 100},
    # longitudes
    {"dataset": "/gt1l/land_ice_segments/longitude", "startrow": 0, "numrows": 100},
    {"dataset": "/gt1r/land_ice_segments/longitude", "startrow": 0, "numrows": 100},
    {"dataset": "/gt2l/land_ice_segments/longitude", "startrow": 0, "numrows": 100},
    {"dataset": "/gt2r/land_ice_segments/longitude", "startrow": 0, "numrows": 100},
    {"dataset": "/gt3l/land_ice_segments/longitude", "startrow": 0, "numrows": 100},
    {"dataset": "/gt3r/land_ice_segments/longitude", "startrow": 0, "numrows": 100}
]

atl06 = h5.h5p(datasets, resource, asset)

In [ ]:
atl06

<a id='atl03p'></a>
### Subset ATL03 photon cloud data

The [`icesat2.atl03sp`](https://slideruleearth.io/web/rtd/api_reference/icesat2.html#atl03sp) function makes an ATL03 subsetting request to SlideRule servers and returns a GeoDataFrame of photons.  Documentation for this function can be found in the [API reference](https://slideruleearth.io/web/rtd/api_reference/icesat2.html#atl03sp).

In the example below, a set of resources is specified via the `poly`, `rgt`, and `cycle` parameters.  Because each granule contains so many photons, it is necessary when making this call to limit the area over which the subsetting request is made, along with the number of granules inside that area. By supplying a GeoJSON file (which is read and processed by the `sliderule.toregion` function into a format usable by SlideRule), the extent of data read in each granule is trimmed.  By supplying a reference ground track (`rgt`), and the cycle number, the number of granules is reduced - in this case to a single granule.

The other parameters in the request are used to specify different aspects of the ATL03 subsetting request.  The `srt` parameters specifies the surface type, which in this case is land. The surface type is used in conjunction with the next parameter - `cnf` which is the confidence level.  A confindence level of high tells SlideRule to only include photons that are highly likely to be surface reflections off of land.  (As a different example, if the `srt` parameter specified land ice and the confidence level was low, then SlideRule would include all photons that had *at least* a low likelihood of being a reflection off of land ice). Lastly, the `len` and `res` parameters specify the length and resolution of the photon segments being returned.  In this case we are asking for 20m segments of photons every 20m.  The length and step size of the segment does not matter so much if it is only photons being returned, but when other processing parameters are supplied (like minimal along track spread of a segment), then it matters more.

Lastly, the call to `icesat2.atl03sp` is made which sends the HTTP request to SlideRule's servers and then waits and accumulates the response from the servers into a GeoDataFrame, which each row representing a single photon.

In [ ]:
# Build Region of Interest
region = sliderule.toregion('grandmesa.geojson')

# Build ATL03 Subsetting Request Parameters
parms = {
    "poly": region["poly"],
    "rgt": 737,
    "cycle": 16,
    "srt": icesat2.SRT_LAND,
    "cnf": icesat2.CNF_SURFACE_HIGH,
    "len": 20.0,
    "res": 20.0
}

# Make ATL03 Subsetting Request
atl03 = icesat2.atl03sp(parms, resources=resources)

In [ ]:
atl03

## Advanced Functionality
- Private clusters
- GeoParquet output to S3
- Customized YAPC classification
- Include ancillary data
- Query CMR, CMR-STAC, and TNM
- Directly sample supported raster datasets
- Subset via rasterized area of interest
- Kmeans clustering